## Data Preprocessing using Dask

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
liar2_train = pd.read_csv("liar_dataset/train.csv")
liar2_train.head()

,id,label,statement,date,subject,speaker,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,context,justification
0,13847,5,"90 percent of Americans ""support universal bac...","October 2, 2017",government regulation;polls and public opinion...,chris abele,"Chris Abele is Milwaukee County Executive, a p...",wisconsin,1,4,5,3,5,2,a tweet,"""Universal"" is the term for background checks ..."
1,13411,1,Last year was one of the deadliest years ever ...,"May 19, 2017",after the fact;congress;criminal justice;histo...,thom tillis,Thom Tillis is a Republican who serves as U.S....,north carolina,0,2,7,3,2,0,a press release supporting the Back The Blue A...,"Sen. Thom Tillis, a North Carolina Republican,..."
2,10882,0,"Bernie Sanders's plan is ""to raise your taxes ...","October 28, 2015",taxes,chris christie,"Chris Christie announced June 6, 2023 that he ...",national,21,20,27,11,17,8,"Boulder, Colo","Christie said that Sanders’s plan is ""to raise..."
3,20697,4,Voter ID is supported by an overwhelming major...,"December 8, 2021",voter id laws,lee zeldin,Lee Zeldin is a Republican representing New Yo...,new york,1,2,0,0,0,0,a Tweet,Zeldin claimed voter identification requiremen...
4,6095,2,"Says Barack Obama ""robbed Medicare (of) $716 b...","August 12, 2012",federal budget;history;medicare;retirement,mitt romney,Mitt Romney is a U.S. senator from Utah. He ra...,national,31,33,58,35,32,19,"an interview on ""60 Minutes""","Romney said, ""There's only one president that ..."


In [ ]:
def convert_boolean_to_int(df):
    for col in df.columns:
        if df[col].dtype == "bool":
            df[col] = df[col].astype(int)
    return df

In [ ]:
import dask.dataframe as dd

# Convert Pandas DataFrame to Dask DataFrame
train_dask = dd.from_pandas(liar2_train, npartitions=4)

train_dask["speaker"] = train_dask["speaker"].fillna("unknown")
train_dask["context"] = train_dask["context"].fillna("unknown")

# Categorize the 'speaker' column
train_dask = train_dask.categorize(columns=["speaker", "context"])

# Perform one-hot encoding
ddf_one_hot = dd.get_dummies(
    train_dask, columns=["speaker", "context"], prefix=["speaker", "context"]
)

# Convert boolean columns to integers
ddf_one_hot = ddf_one_hot.map_partitions(convert_boolean_to_int)
# print(ddf_one_hot.dtypes)
# Converting back to Pandas
df_one_hot = ddf_one_hot.compute()

# Display the result
df_one_hot.head()

,id,label,statement,date,subject,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,justification,speaker_ afscme,speaker_ infowars,speaker_ my city bikes,speaker_ north carolina democratic party,speaker_ stokes county school teacher,speaker_ stop deceptive amendments,speaker_ young americans for liberty,"speaker_""watch the water""",speaker_10 gop louisiana lawmakers,speaker_13th district gop slate,speaker_18% of the american public,speaker_60 plus association,speaker_a better wisconsin together,speaker_a.j. jacobs,speaker_aaron degroot,speaker_aaron lieberman,speaker_aaron rodgers,speaker_abc news-us,speaker_abel collins,speaker_abigail spanberger,speaker_aclu foundation of georgia,speaker_aclu of north carolina,speaker_act for america,speaker_actionaid uk,speaker_adam hasner,speaker_adam kinzinger,speaker_adam laxalt,speaker_adam putnam,speaker_adam schefter,speaker_adam schiff,speaker_adam smith,speaker_addicting information,speaker_adrian fontes,speaker_adrian garcia,speaker_adriano espaillat,speaker_advancing wisconsin,speaker_afl-cio,speaker_afl-cio of new jersey,speaker_afscme people,speaker_aftab pureval,speaker_ainsley earhardt,speaker_al cardenas,speaker_al franken,speaker_al gore,speaker_al hunt,speaker_al roker,speaker_al sharpton,speaker_alabama's future,speaker_alan bates,speaker_alan essig,speaker_alan grayson,speaker_alan hays,speaker_alan krueger,speaker_alan powell,speaker_alan simpson,speaker_albert papp jr.,speaker_alberta darling,speaker_albio sires,speaker_alcee hastings,speaker_alejandro mayorkas,speaker_alex azar,speaker_alex berenson,speaker_alex castellanos,speaker_alex epstein,speaker_alex jones,speaker_alex lasry,speaker_alex mcmurtrie jr.,speaker_alex mooney,speaker_alex padilla,speaker_alex sink,speaker_alex wan,speaker_alexander snitker,speaker_alexander strenger,speaker_alexandria ocasio-cortez,speaker_alexi giannoulias,speaker_alfonso lopez,speaker_alicia menendez,speaker_alisha thomas morgan,speaker_alison littell mchose,speaker_alison lundergan grimes,speaker_alissa keny-guyer,speaker_allan fung,speaker_allan levene,speaker_allan ritter,speaker_allen alley,speaker_allen andrews,speaker_allen boyd,speaker_allen buckley,speaker_allen frances,speaker_allen peake,speaker_allen thomas,speaker_allen west,speaker_alliance defending freedom,speaker_alliance for america's future,speaker_alliance for retired americans,speaker_allison tant,speaker_alma adams,speaker_alternativepac,speaker_alyssa milano,speaker_amalgamated transit union,speaker_amanda carpenter,speaker_amanda chase,speaker_amanda fritz,speaker_amardeep kaleka,speaker_america first,speaker_america first legal,speaker_america today,speaker_america's health insurance plans,speaker_america's natural gas alliance,speaker_american action network,speaker_american automobile association,speaker_american beverage association,speaker_american bird conservancy,speaker_american bridge 21st century,speaker_american civil liberties union,speaker_american civil liberties union of florida,speaker_american commitment,speaker_american crossroads,speaker_american energy alliance,speaker_american federation for children,speaker_american future fund,speaker_american international group,speaker_american israel public affairs committee,speaker_american leadership action,speaker_american leadhership pac,speaker_american life league,speaker_american petroleum institute,speaker_american postal workers union,speaker_american principles project,speaker_american prosperity alliance,speaker_american revolution,speaker_american unity pac,speaker_americanjournalreview.com,speaker_americannews.com,speaker_americanproud.info,speaker_americans against food taxes,speaker_americans for a strong defense,speaker_americans for common sense solutions,speaker_americans for job security,speaker_americans for prosperity,speaker_americans for prosperity georgia chapter,"speaker_americans for prosperi